In [1]:
import numpy as np
import h5py
import pandas as pd

f_snap = '/home/jovyan/Data/Sims/IllustrisTNG/CV_0/snap_033.hdf5'
data = h5py.File(f_snap, 'r')

data

<HDF5 file "snap_033.hdf5" (mode r)>

In [2]:
import sys
import illustris_python.illustris_python as il

basePath = '/home/jovyan/Data/Sims/IllustrisTNG/1P_0/'
header = il.groupcat.loadHeader(basePath, 33)

little_h = 0.6774
scaling_factor = header['Time']


In [3]:

fields = ['SubhaloMass','SubfindID','SnapNum']
tree = il.sublink.loadTree(basePath, 33, 0, fields=None) 

In [4]:
halos = il.groupcat.loadHalos(basePath, 33, fields=['GroupNsubs', 'GroupFirstSub', 'GroupLenType'])
halos['HaloStarsLen'] = halos['GroupLenType'][:, 4]
del halos['GroupLenType']

halos_df = pd.DataFrame(halos)
halos_df['HaloID'] = halos_df.index
halos_df

,count,GroupNsubs,GroupFirstSub,HaloStarsLen,HaloID
0,14857,262,0,106102,0
1,14857,25,262,7556,1
2,14857,29,287,4872,2
3,14857,24,316,7519,3
4,14857,21,340,3381,4
...,...,...,...,...,...
14852,14857,1,11137,0,14852
14853,14857,1,11138,0,14853
14854,14857,0,-1,0,14854
14855,14857,0,-1,0,14855


In [5]:
GroupFirstSub = il.groupcat.loadHalos(basePath, 33, fields=['GroupFirstSub'])

print(GroupFirstSub)

fields = ['SubhaloMassType','SubhaloSFRinRad', 'SubhaloPos', 
          'SubhaloHalfmassRadType', 'SubhaloSpin', 'SubhaloLenType']
subhalos = il.groupcat.loadSubhalos(basePath, 33, fields=fields)

subhalos['SubhaloMassStar'] = subhalos['SubhaloMassType'][:, 4] 
subhalos['SubhaloMassGas'] = subhalos['SubhaloMassType'][:, 0] * 1e10 / little_h
subhalos['SubhaloMassBH'] = subhalos['SubhaloMassType'][:, 5] * 1e10 / little_h
subhalos['SubhaloMassDM'] = subhalos['SubhaloMassType'][:, 1] * 1e10 / little_h
subhalos['SubhaloSpin_x'] = subhalos['SubhaloSpin'][:, 0] / little_h
subhalos['SubhaloSpin_y'] = subhalos['SubhaloSpin'][:, 1] / little_h
subhalos['SubhaloSpin_z'] = subhalos['SubhaloSpin'][:, 2] / little_h
subhalos['SubhaloPos_x'] = subhalos['SubhaloPos'][:, 0] * scaling_factor / little_h
subhalos['SubhaloPos_y'] = subhalos['SubhaloPos'][:, 1] * scaling_factor / little_h
subhalos['SubhaloPos_z'] = subhalos['SubhaloPos'][:, 2] * scaling_factor / little_h
subhalos['SubhaloStarsLen'] = subhalos['SubhaloLenType'][:, 4]
subhalos['SubhaloGasLen'] = subhalos['SubhaloLenType'][:, 0]

subhalos['SubhaloHalfmassRadStar'] = subhalos['SubhaloHalfmassRadType'][:, 4] * scaling_factor / little_h

del subhalos['SubhaloSpin']
del subhalos['SubhaloMassType']
del subhalos['SubhaloHalfmassRadType']
del subhalos['SubhaloPos']
del subhalos['SubhaloLenType']
del subhalos['count']

tng_df = pd.DataFrame(subhalos)
tng_df['GalaxyID'] = tng_df.index
tng_df['Central'] = tng_df['GalaxyID'].isin(GroupFirstSub)
#tng_df = tng_df[tng_df.SubhaloMassStar > 0]
#tng_df = tng_df[tng_df.SubhaloHalfmassRadStar > 0]
tng_df

#tng_df.filter(items=range(262), axis=0)['SubhaloStarsLen'].sum()

[  0 262 287 ...  -1  -1  -1]


,SubhaloSFRinRad,SubhaloMassStar,SubhaloMassGas,SubhaloMassBH,SubhaloMassDM,SubhaloSpin_x,SubhaloSpin_y,SubhaloSpin_z,SubhaloPos_x,SubhaloPos_y,SubhaloPos_z,SubhaloStarsLen,SubhaloGasLen,SubhaloHalfmassRadStar,GalaxyID,Central
0,4.696040,64.474846,8.094857e+12,1.923114e+09,1.826850e+13,-3031.067627,4900.544434,4987.395996,9266.201172,15039.223633,4439.000488,73999,412593,51.741257,0,True
1,0.000000,3.567838,0.000000e+00,2.405884e+08,1.529249e+11,53.794422,-49.282639,-293.973053,10220.753906,14546.266602,4472.988770,4039,0,8.058429,1,False
2,1.609842,2.858819,2.742845e+10,1.466575e+08,7.967355e+10,215.432373,9.950420,10.984626,8748.359375,15369.004883,4338.590332,3153,1231,6.806039,2,False
3,0.000000,2.078028,0.000000e+00,1.756334e+08,9.966005e+10,57.604462,142.656601,34.921692,9828.577148,14768.825195,5114.464844,2366,0,4.113121,3,False
4,0.000000,2.045388,0.000000e+00,1.285266e+08,6.295650e+10,195.902649,161.889328,-100.465813,9201.371094,15559.604492,4283.999023,2223,0,6.897761,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11134,0.000000,0.000000,0.000000e+00,0.000000e+00,4.670652e+08,2.189512,10.462361,5.735119,31104.496094,29906.201172,18393.292969,0,0,0.000000,11134,True
11135,0.000000,0.000000,0.000000e+00,0.000000e+00,4.670652e+08,8.397372,-19.229832,-8.930002,29873.419922,33386.289062,19247.484375,0,0,0.000000,11135,True
11136,0.000000,0.000000,0.000000e+00,0.000000e+00,4.865263e+08,0.878831,19.551769,25.607111,29868.318359,31264.892578,22733.425781,0,0,0.000000,11136,True
11137,0.000000,0.000000,0.000000e+00,0.000000e+00,4.865263e+08,25.094234,50.454872,-71.994118,33552.382812,34391.164062,25816.304688,0,0,0.000000,11137,True


In [6]:
tree

{'count': 2786,
 'DescendantID': array([-1,  0,  1, ...,  0,  0,  0]),
 'FirstProgenitorID': array([ 1,  2,  3, ..., -1, -1, -1]),
 'FirstSubhaloInFOFGroupID': array([0, 1, 2, ..., 1, 1, 1]),
 'GroupBHMass': array([0.17643267, 0.17500821, 0.17316236, ..., 0.17500821, 0.17500821,
        0.17500821], dtype=float32),
 'GroupBHMdot': array([0.00305982, 0.00102392, 0.00649596, ..., 0.00102392, 0.00102392,
        0.00102392], dtype=float32),
 'GroupCM': array([[ 6374.9536, 10151.386 ,  3039.1387],
        [ 6369.948 , 10147.439 ,  3046.5159],
        [ 6368.582 , 10140.973 ,  3059.8032],
        ...,
        [ 6369.948 , 10147.439 ,  3046.5159],
        [ 6369.948 , 10147.439 ,  3046.5159],
        [ 6369.948 , 10147.439 ,  3046.5159]], dtype=float32),
 'GroupFirstSub': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'GroupGasMetalFractions': array([[7.5495434e-01, 2.4251132e-01, 2.5414067e-04, ..., 1.1570062e-04,
         1.3884623e-04, 7.6709955e-05],
        [7.5488007e-01, 2.4255045e-01,

In [11]:
numMergers = il.sublink.numMergers(tree, minMassRatio=0.1)

In [12]:
numMergers

6

In [13]:
fields = ['Masses']
fields = ['ParticleIDs','GFM_StellarFormationTime', 'Masses',]
stars = il.snapshot.loadSubset(basePath, 33, 'stars', fields=fields)
stars_df = pd.DataFrame(stars)

sid = 0
stars_df = stars_df.iloc[tng_df['SubhaloStarsLen'][:sid].sum():
                         tng_df['SubhaloStarsLen'][:sid+1].sum()]
print(stars_df)

stars_df = stars_df[stars_df.GFM_StellarFormationTime > 0]

print(stars_df)
a = stars_df.Masses.sum()
print(a,  tng_df['SubhaloMassStar'][sid])
b = a * 1e10 / little_h

from decimal import Decimal

'%.5E' % Decimal(b)

        count  ParticleIDs  GFM_StellarFormationTime    Masses
0      202437     37613530                  0.496575  0.000715
1      202437     36797750                  0.446528  0.000784
2      202437     37622635                  0.496603  0.000795
3      202437     35251559                  0.355481  0.000715
4      202437     34092152                  0.254945  0.000717
...       ...          ...                       ...       ...
73994  202437     36869150                  0.478392  0.000901
73995  202437     35959604                  0.472163  0.000545
73996  202437      2753817                  0.588547  0.000910
73997  202437     37699664                  0.507845  0.000642
73998  202437     37697875                  0.605744  0.000961

[73999 rows x 4 columns]
        count  ParticleIDs  GFM_StellarFormationTime    Masses
0      202437     37613530                  0.496575  0.000715
1      202437     36797750                  0.446528  0.000784
2      202437     37622635   

'9.51729E+11'

In [16]:
stars = il.snapshot.loadSubhalo(basePath, 33, 65,
                        'stars', fields=["ParticleIDs", "GFM_StellarFormationTime"])

In [17]:
stars

{'count': 143,
 'ParticleIDs': array([43324284, 43384196, 42803918, 43143206, 42504798, 43028070,
        43245583, 43339152, 43165921, 43184430, 42744141, 43171669,
        43170743, 43396275, 43171702, 43411056, 42761732, 42566883,
        42963325, 43383788, 43099198, 42721183, 42718904, 43165935,
        43165001, 42749350, 42532138, 43481561, 43109013, 42660920,
        43365474, 43226971, 42408830, 42453400, 42548347, 42651477,
        42943826, 42661914, 43193556, 43302729, 41091190, 41792696,
        42647799, 43634695, 43008948, 43132913, 43068939, 43145708,
        42692312, 43267615, 43246390, 43076728, 42729550, 42609285,
        42918143, 43240477, 42559173, 43255212, 43324092, 42931996,
        43422283, 43041491, 42728202, 43266899, 42709560, 43668414,
        43336101, 43268486, 43118346, 42394494, 42602053, 42921006,
        43414231, 42294371, 43287439, 43044589, 41992805, 43127657,
        43179311, 42688752, 43320248, 43360326, 42598040, 43019394,
        43676966, 